---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 
          'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 
          'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 
          'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 
          'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 
          'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 
          'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 
          'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 
          'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 
          'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 
          'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 
          'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 
          'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 
          'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
import re

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    ''' read university_towns.txt file
    1. the line end with '[edit]' is the Name of State
    2. the followed lines are the college towns in the state
    Function get_list_of_university_towns was answered incorrectly, 0.1 points were not awarded. 
    ''' 
    state_p = re.compile(r"\[edit\]")
    region_p1 = re.compile(r"\s\(")  # regular cases 
    region_p2 = re.compile(r":$")            # end with ':'
    with open('university_towns.txt', encoding='utf-8') as fd:
        lines = fd.readlines()
        
        state_region = []
        cnt = 0
        for line in lines:
            line = line.rstrip()
            if '[edit]' in line: 
                # extract state info
                state = re.sub(state_p, "", line)
            else: 
                # extract region info
                region = None
                if ' (' in line:
                    # strip from " (" to end of line w/ position index
                    region = line[0:re.search(region_p1, line).start()]
#                 elif ':' in line:
#                     # strip from ":" to end of line
#                     region = line[0:re.search(region_p2, line).start() + 1]
                else: 
                    region = line[0:]
                    
#                 if cnt in [179, 184]:
#                     print("line({}): {} w/ len= {} --> [{}, {}]".format(cnt, line, len(line), state, region))
#                 cnt += 1
                state_region.append([state, region])
    
    df = pd.DataFrame(state_region, columns=["State", "RegionName"]  )
    
    return df


# get_list_of_university_towns()

In [ ]:
import re
import pandas as pd
import numpy as np

def test():
    # list of unique states
    stateStr = """
    Ohio, Kentucky, American Samoa, Nevada, Wyoming
    ,National, Alabama, Maryland, Alaska, Utah
    ,Oregon, Montana, Illinois, Tennessee, District of Columbia
    ,Vermont, Idaho, Arkansas, Maine, Washington
    ,Hawaii, Wisconsin, Michigan, Indiana, New Jersey
    ,Arizona, Guam, Mississippi, Puerto Rico, North Carolina
    ,Texas, South Dakota, Northern Mariana Islands, Iowa, Missouri
    ,Connecticut, West Virginia, South Carolina, Louisiana, Kansas
    ,New York, Nebraska, Oklahoma, Florida, California
    ,Colorado, Pennsylvania, Delaware, New Mexico, Rhode Island
    ,Minnesota, Virgin Islands, New Hampshire, Massachusetts, Georgia
    ,North Dakota, Virginia
    """
    #list of regionName entries string length
    regNmLenStr = """
    06,08,12,10,10,04,10,08,09,09,05,06,11,06,12,09,08,10,12,06,
    06,06,08,05,09,06,05,06,10,28,06,06,09,06,08,09,10,35,09,15,
    13,10,07,21,08,07,07,07,12,06,14,07,08,16,09,10,11,09,10,06,
    11,05,06,09,10,12,06,06,11,07,08,13,07,11,05,06,06,07,10,08,
    11,08,13,12,06,04,08,10,08,07,12,05,06,09,07,10,16,10,06,12,
    08,07,06,06,06,11,14,11,07,06,06,12,08,10,11,06,10,14,04,11,
    18,07,07,08,09,06,13,11,12,10,07,12,07,04,08,09,09,13,08,10,
    16,09,10,08,06,08,12,07,11,09,07,09,06,12,06,09,07,10,09,10,
    09,06,15,05,10,09,11,12,10,10,09,13,06,09,11,06,11,09,13,37,
    06,13,06,09,49,07,11,12,09,11,11,07,12,10,06,06,09,04,09,15,
    10,12,05,09,08,09,09,07,14,06,07,16,12,09,07,09,06,32,07,08,
    08,06,10,36,09,10,09,06,09,11,09,06,10,07,14,08,07,06,10,09,
    05,11,07,06,08,07,05,07,07,04,06,05,09,04,25,06,07,08,05,08,
    06,05,11,09,07,07,06,13,09,05,16,05,10,09,08,11,06,06,06,10,
    09,07,06,07,10,05,08,07,06,08,06,30,09,07,06,11,07,12,08,09,
    16,12,11,08,06,04,10,10,15,05,11,11,09,08,06,04,10,10,07,09,
    11,08,26,07,13,05,11,03,08,07,06,05,08,13,10,08,08,08,07,07,
    09,05,04,11,11,07,06,10,11,03,04,06,06,08,08,06,10,09,05,11,
    07,09,06,12,13,09,10,11,08,07,07,08,09,10,08,10,08,56,07,12,
    07,16,08,04,10,10,10,10,07,09,08,09,09,10,07,09,09,09,12,14,
    10,29,19,07,11,12,13,13,09,10,12,12,12,08,10,07,10,07,07,08,
    08,08,09,10,09,11,09,07,09,10,11,11,10,09,09,12,09,06,08,07,
    12,09,07,07,06,06,08,06,15,08,06,06,10,10,10,07,05,10,07,11,
    09,12,10,12,04,10,05,05,04,14,07,10,09,07,11,10,10,10,11,15,
    09,14,12,09,09,07,12,04,10,10,06,10,07,28,06,10,08,09,10,10,
    10,13,12,08,10,09,09,07,09,09,07,11,11,13,08,10,07
    """

    df = get_list_of_university_towns()

    cols = ["State", "RegionName"]

    print('Shape test: ', "Passed" if df.shape ==
          (517, 2) else 'Failed')
    print('Index test: ',
          "Passed" if df.index.tolist() == list(range(517))
          else 'Failed')

    print('Column test: ',
          "Passed" if df.columns.tolist() == cols else 'Failed')
    print('\\n test: ',
          "Failed" if any(df[cols[0]].str.contains(
              '\n')) or any(df[cols[1]].str.contains('\n'))
          else 'Passed')
    print('Trailing whitespace test:',
          "Failed" if any(df[cols[0]].str.contains(
              '\s+$')) or any(df[cols[1]].str.contains(
                  '\s+$'))
          else 'Passed')
    print('"(" test:',
          "Failed" if any(df[cols[0]].str.contains(
              '\(')) or any(df[cols[1]].str.contains(
                  '\('))
          else 'Passed')
    print('"[" test:',
          "Failed" if any(df[cols[0]].str.contains(
              '\[')) or any(df[cols[1]].str.contains(
                  '\]'))
          else 'Passed')

    states_vlist = [st.strip() for st in stateStr.split(',')]

    mismatchedStates = df[~df['State'].isin(
        states_vlist)].loc[:, 'State'].unique()
    print('State test: ', "Passed" if len(
        mismatchedStates) == 0 else "Failed")
    if len(mismatchedStates) > 0:
        print()
        print('The following states failed the equality test:')
        print()
        print('\n'.join(mismatchedStates))

    df['expected_length'] = [int(s.strip())
                             for s in regNmLenStr.split(',')
                             if s.strip().isdigit()]
    regDiff = df[df['RegionName'].str.len() != df['expected_length']].loc[
        :, ['RegionName', 'expected_length']]
    regDiff['actual_length'] = regDiff['RegionName'].str.len()
    print('RegionName test: ', "Passed" if len(regDiff) ==
          0 else ' \nMismatching regionNames\n {}'.format(regDiff))
    
test()

In [19]:
import re
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 500)

def load_gdp_quarter():
    df = pd.read_excel("gdplev.xls", skiprows=5)
    
    # delete rows and starts from 2000q1
    df = df.iloc[214:]
    
    # remove unnecessary columns
    cols = ["Unnamed: 4", "GDP in billions of current dollars.1", 'GDP in billions of chained 2009 dollars.1']
    col_lbl = {"Unnamed: 4": 'Quarter', "GDP in billions of current dollars.1": "GDP1", 
               'GDP in billions of chained 2009 dollars.1': "GDP"}
    df = df[cols]
    
    df.rename(index=str, columns=col_lbl, inplace=True)
    df.index = [idx for idx in range(len(df))]
    
    return df
    
def quarter_diff(df):
    
    gdp = df['GDP']
    
    gdp_diff = []
    for idx, val in enumerate(gdp.values):
        if idx == 0:
            gdp_diff.append(np.NaN)
        else:
            gdp_diff.append(val - gdp.values[idx - 1])
            
    df['Diff'] = pd.Series(gdp_diff)
    
    return df

def recession_assign(df, debug = False):
    """
    Return a dataframe w/ recession column
    """
    neg_cnt, pos_cnt = 0, 0
    res_mask = []
    recession = False
    for val in df['Diff'].values:
        
        if debug:
            print("Value= {:8.2f}".format(val), end=" ")
            
        if np.isnan(val):
            res_mask.append(np.NaN)
            if debug:
                print("0 - rec= {:5s} neg_cnt= {} pos_cnt= {} res_mask({:3d})= {}".format(
                    str(recession), neg_cnt, pos_cnt, len(res_mask)-1, res_mask[-2:]))
            continue

        if recession:                               # recession = True
            if val < 0:                             # recession continued
                neg_cnt, pos_cnt = 0, 0             # reset counters
                res_mask.append(True)
                if debug:
                    print("1 - rec= {:5s} neg_cnt= {} pos_cnt= {} res_mask({:3d})= {}".format(
                        str(recession), neg_cnt, pos_cnt, len(res_mask)-1, res_mask[-2:]))
            elif (val > 0 and pos_cnt == 0):          # possible end of recession starts
                pos_cnt += 1
                neg_cnt = 0
                res_mask.append(True)
                if debug:
                    print("2 - rec= {:5s} neg_cnt= {} pos_cnt= {} res_mask({:3d})= {}".format(
                        str(recession), neg_cnt, pos_cnt, len(res_mask)-1, res_mask[-2:]))
            elif (val > 0 and pos_cnt == 1):          # end of recession ensure
                pos_cnt, neg_cnt = 0, 0
                res_mask.append(True)
                recession = False
                if debug:
                    print("3 - rec= {:5s} neg_cnt= {} pos_cnt= {} res_mask({:3d})= {}".format(
                        str(recession), neg_cnt, pos_cnt, len(res_mask)-1, res_mask[-2:]))
            else: 
                # should not be here - for warning purpose
                print("1 - Rec= {:5s} val= {} pos_cnt= {} neg_cnt= {:3d}".format(
                        recession, val, pos_cnt, neg_cnt))
                exit(1)
        else:                                      # recession = False
            if val > 0:                            # regular period continued
                neg_cnt, pos_cnt = 0, 0
                res_mask.append(False)
                if debug:
                    print("4 - rec= {:5s} neg_cnt= {} pos_cnt= {} res_mask({:3d})= {}".format(
                        str(recession), neg_cnt, pos_cnt, len(res_mask)-1, res_mask[-2:]))
            elif (val < 0 and neg_cnt == 0):         # possible start of recession
                neg_cnt += 1
                pos_cnt = 0
                res_mask.append(False)
                if debug:
                    print("5 - rec= {:5s} neg_cnt= {} pos_cnt= {} res_mask({:3d})= {}".format(
                        str(recession), neg_cnt, pos_cnt, len(res_mask)-1, res_mask[-2:]))
            elif (val < 0 and neg_cnt == 1):         # ensure resession start
                neg_cnt, pos_cnt = 0, 0
                res_mask[-1] = True
                res_mask.append(True)
                recession = True
                if debug:
                    print("6 - rec= {:5s} neg_cnt= {} pos_cnt= {} res_mask({:3d})= {}".format(
                        str(recession), neg_cnt, pos_cnt, len(res_mask)-1, res_mask[-2:]))
            else: 
                # should not be here - for warning purpose
                print("2 - Rec= {:5s} val= {} pos_cnt= {} neg_cnt= {}".format(
                        recession, val, pos_cnt, neg_cnt))
                exit(1)
        
#     print(res_mask)
        
    df['Recession'] = pd.Series(res_mask)
    
    df.index = df['Quarter']
    
    return df

def recession_start_end(df):
    start, end, bottom = [], [], []
    
    match = [idx for idx, val in enumerate(df['Recession']) if val == True]
    
    for idx, val in enumerate(match):
        if (idx == 0):
            start.append(df.iloc[val].name)
        elif (idx == len(match) - 1):
            end.append(df.iloc[val].name)
            bottom.append(df.iloc[val-2].name)
        elif val > match[idx - 1] + 1:
            end.append(df.iloc[match[idx-1]].name)
            bottom.append(df.iloc[match[idx-3].name])
            start.append(df.iloc[val].name)
                
#     print("Match: {}, start= {} end= {} bottom= {}".format(match, start, end, bottom))
    
    return start, end, bottom
    
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    df = load_gdp_quarter()
    df = quarter_diff(df)
    df = recession_assign(df, debug=False)
    
    start, end, bottom = recession_start_end(df)

    return start[0]
#     return df

# load_gdp()
# get_recession_start()

In [11]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''

    df = load_gdp_quarter()
    df = quarter_diff(df)
    df = recession_assign(df, debug=False)
    
    start, end, bottom = recession_start_end(df)

    return end[0]

# get_recession_end()

In [22]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''

    df = load_gdp_quarter()
    df = quarter_diff(df)
    df = recession_assign(df, debug=False)
    
    start, end, bottom = recession_start_end(df)
    
#     print(bottom)

    return bottom[0]

# get_recession_bottom()

In [23]:
import re
import pandas as pd
import numpy as np

"""
Discussion Forum: 
https://www.coursera.org/learn/python-data-analysis/discussions/weeks/4/threads/nB8Rqba_EeeVOgq1E_eRMA

This is because the dataframe can have duplicate index. the aim of the question is to group each 3 
columns corresponding to a quarter together without affecting the rows

The trick to solving this question efficiently, is to the use the concept of time series and resample 
that was introduced in week 3 demo lecture Date Functionality

The difference from the example that was introduced in the lecture is that we have 6 information column

here is how to go about applying resample

1. housing = read the csv file 'City_Zhvi_AllHomes.csv' (from the online platform or the downloaded 
   version from google drive, do not download the file from the Zillow website)
2. use the states dictionary provided at the beginning of the notebook to map the column State values 
   from abbreviations to names.
3. housing = set the columns ['State','RegionName'] as index of housing
4. housing = select the columns 2000-01-2016-08 from the data-frame housing
5. housing = convert the column names from strings to date-time.
6. use resample on on the columns of housing i.e.
    housing = housing.resample('Q',axis=1).mean()
7. make sure you convert the column names from date-time back to strings as per the docstring, an 
    example of how to do so is
    housing = housing.rename(columns=lambda x: str(x.to_period('Q')).lower())

Ref: https://stackoverflow.com/questions/41661451/pandas-compute-the-half-yearly-average-of-monthly-collected-house-sales
"""

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # read data in 
    housing = pd.read_csv("City_Zhvi_AllHomes.csv")
       
    # map the state abbreviation to State names
    housing['State'] = housing['State'].apply(lambda x: states[x])
    
    # set the columns ['State','RegionName'] as index of housing
    housing.index = housing[['State', 'RegionName']]

    # select the columns 2000-01-2016-08 from the data-frame housing
    housing = housing.loc[:,'2000-01':]

#     cols = ['RegionID', 'RegionName', 'State', 'Metro', 'CountyName', 'SizeRank']
#     cols = []
#     for year in range(2000, 2017):
#         for mon in range(1, 13):
#             if year < 2016:
#                 cols.append(str(year) + "-" + "{:02d}".format(mon))
#             elif year == 2016 and mon < 9:
#                 cols.append(str(year) + "-" + "{:02d}".format(mon))
#              
#     housing = housing[cols]

    # convert the column names from strings to date-time
    housing.columns = pd.to_datetime(housing.columns)
    
    # use resample on on the columns of housing
    housing = month_to_quater(housing)
        
#     print(df.columns)
    
    return housing


convert_housing_data_to_quarters()

,2000-01-01 00:00:00,2000-02-01 00:00:00,2000-03-01 00:00:00,2000-04-01 00:00:00,2000-05-01 00:00:00,2000-06-01 00:00:00,2000-07-01 00:00:00,2000-08-01 00:00:00,2000-09-01 00:00:00,2000-10-01 00:00:00,...,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,2016-07-01 00:00:00,2016-08-01 00:00:00
"(New York, New York)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
"(California, Los Angeles)",204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
"(Illinois, Chicago)",136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
"(Pennsylvania, Philadelphia)",52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
"(Arizona, Phoenix)",111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
"(Nevada, Las Vegas)",131700.0,132600.0,133500.0,134100.0,134400.0,134600.0,134800.0,135400.0,136000.0,136400.0,...,190600,192000,193600,194800,195400,196100,197300,198200,199300,200600
"(California, San Diego)",219200.0,222900.0,226600.0,230200.0,234400.0,238500.0,242000.0,245300.0,249000.0,252500.0,...,525700,526700,527800,529200,531000,533900,536900,537900,539000,540500
"(Texas, Dallas)",85100.0,84500.0,83800.0,83600.0,83800.0,84200.0,84500.0,84900.0,85200.0,86000.0,...,134600,136600,138700,140600,142200,143300,144500,146000,148200,150400
"(California, San Jose)",364100.0,374000.0,384700.0,395700.0,407100.0,416900.0,424700.0,431700.0,439200.0,447600.0,...,789700,792100,795800,803100,811900,817600,819100,820100,821700,822700
"(Florida, Jacksonville)",88000.0,88800.0,89000.0,88900.0,89600.0,90600.0,91200.0,91700.0,92200.0,92600.0,...,132000,132500,133100,133900,134900,136000,137200,138400,139500,140300


In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"